In [1]:
import os
import requests

import pandas as pd
import simplekml
import collada_wt

from dotenv import load_dotenv

import requests
import json
from types import SimpleNamespace

import pickle
import os
import re

import openstreetmap_mapping as osm

import pandas as pd
import numpy as np

from bokeh.plotting import show, output_file
from bokeh.io import output_notebook

from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, OpenURL, TapTool
from bokeh.models import WMTSTileSource
from bokeh.models import HoverTool
from bokeh.models import LabelSet
from bokeh.palettes import viridis, Category20, Blues256

output_notebook()

from pyproj import Transformer

Loading BokehJS ...

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# Use pyproj to transform longitude and latitude into web-mercator and add to a copy of the asset dataframe
TRANSFORM_4326_TO_3857 = Transformer.from_crs("EPSG:4326", "EPSG:3857")
TRANSFORM_3857_TO_4326 = Transformer.from_crs("EPSG:3857", "EPSG:4326")

In [4]:
class Tags(object):
    
    def __init__(self, *initial_data, **kwargs):
              
        for dictionary in initial_data:
            
            for key in dictionary:

                if isinstance(dictionary[key], dict):
                    if len(dictionary[key])>1:
                        setattr(self, 'value', key)
                        setattr(self, key, Tags(dictionary[key]))
                    else:
                        setattr(self, key, Tags(dictionary[key]))
                else:
                    setattr(self, key, dictionary[key])
            
    def __repr__(self):
        return self.value.replace("__",":")
    
    def __str__(self):
        return self.value.replace("__",":")

In [5]:

file_name = 'data/kvs'

if os.path.isfile(file_name):
    # open a file, where you stored the pickled data
    file = open(file_name, 'rb')

    # dump information to that file
    kvs = pickle.load(file)

    # close the file
    file.close()

else:
    kvs = osm.toolkit.get_osm_kvs()  
        
    # open a file, where you ant to store the data
    file = open(file_name, 'wb')

    # dump information to that file
    pickle.dump(kvs, file)

    # close the file
    file.close()


In [6]:
tags = Tags(kvs)

In [7]:
area_Whitelee = "(55.5, -4.4364, 55.7591, -4.1)"
area_California_windRush = "(37,-123,38,-121)"
area_Greece = "(37, 22, 39, 24)"

In [8]:
df_turbines = osm.toolkit.get_osm_data(key=tags.generator__method,tag=tags.generator__method.wind_turbine,area=area_Greece,output="center")


In [9]:
df_turbines_expanded = df_turbines.rename(columns=lambda x: re.sub(':','_',x))
df_turbines_expanded = df_turbines_expanded.rename(columns=lambda x: re.sub('tags.','',x))

required_columns = set(["source","generator_output_electricity","start_date","height_hub","generator_type","rotor_diameter","manufacturer","model"])
missing_columns = list(required_columns - set(df_turbines_expanded.columns))

In [10]:
df_turbines_expanded[missing_columns] = np.nan

In [11]:
df_turbines_expanded


,type,id,lat,lon,tags,generator_method,generator_output_electricity,generator_source,generator_type,manufacturer,...,rotor_diameter,site,generator_manufacturer,generator_model,material,manufacturer_type,colour,key,tag,start_date
0,NaN,1627619830,37.484769,23.156709,"{'generator:method': 'wind_turbine', 'generato...",wind_turbine,3000 kW,wind,horizontal_axis,Vestas,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,generator:method,wind_turbine,NaN
1,NaN,1627619839,37.490580,23.171804,"{'generator:method': 'wind_turbine', 'generato...",wind_turbine,3000 kW,wind,horizontal_axis,Vestas,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,generator:method,wind_turbine,NaN
2,NaN,1627619841,37.490902,23.168938,"{'generator:method': 'wind_turbine', 'generato...",wind_turbine,3000 kW,wind,horizontal_axis,Vestas,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,generator:method,wind_turbine,NaN
3,NaN,1627619845,37.491792,23.167010,"{'generator:method': 'wind_turbine', 'generato...",wind_turbine,3000 kW,wind,horizontal_axis,Vestas,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,generator:method,wind_turbine,NaN
4,NaN,1627619870,37.492753,23.163830,"{'generator:method': 'wind_turbine', 'generato...",wind_turbine,3000 kW,wind,horizontal_axis,Vestas,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,generator:method,wind_turbine,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,NaN,11331002209,38.314829,22.988750,"{'generator:method': 'wind_turbine', 'generato...",wind_turbine,yes,wind,horizontal_axis,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,generator:method,wind_turbine,NaN
610,NaN,11331002210,38.296936,22.953277,"{'generator:method': 'wind_turbine', 'generato...",wind_turbine,yes,wind,horizontal_axis,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,generator:method,wind_turbine,NaN
611,NaN,11331015723,38.320252,22.925420,"{'generator:method': 'wind_turbine', 'generato...",wind_turbine,yes,wind,horizontal_axis,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,generator:method,wind_turbine,NaN
612,NaN,11331015724,38.319960,22.922252,"{'generator:method': 'wind_turbine', 'generato...",wind_turbine,yes,wind,horizontal_axis,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,generator:method,wind_turbine,NaN


In [12]:
df_turbines_expanded["data_rank_colour"] = "sienna"
df_turbines_expanded["data_rank"] = "bronze"

In [13]:
gold_data_list = ["source","generator_output_electricity","start_date","height_hub","generator_type","rotor_diameter","manufacturer","model"]
silver_data_list = ["height_hub","rotor_diameter"]
bronze_data_list = ["lat","lon","id","x","y","coordinates","data_rank","data_rank_colour"]

In [14]:
df_turbines_expanded[silver_data_list].isnull().sum(axis=1)

df_turbines_expanded.loc[df_turbines_expanded[silver_data_list].isnull().sum(axis=1)==0,"data_rank_colour"] = "silver"
df_turbines_expanded.loc[df_turbines_expanded[silver_data_list].isnull().sum(axis=1)==0,"data_rank"] = "silver"

In [15]:
df_turbines_expanded[gold_data_list].isnull().sum(axis=1)

df_turbines_expanded.loc[df_turbines_expanded[gold_data_list].isnull().sum(axis=1)==0,"data_rank_colour"] = "gold"
df_turbines_expanded.loc[df_turbines_expanded[gold_data_list].isnull().sum(axis=1)==0,"data_rank"] = "gold"

In [16]:
df_turbines_expanded["x"], df_turbines_expanded["y"] = TRANSFORM_4326_TO_3857.transform(df_turbines_expanded["lat"], df_turbines_expanded["lon"])
df_turbines_expanded["coordinates"] = tuple(zip(df_turbines_expanded["lat"], df_turbines_expanded["lon"]))

df_turbines_expanded = df_turbines_expanded.rename(columns=lambda x: re.sub(':','_',x))

df_turbines_short = df_turbines_expanded[gold_data_list+bronze_data_list]
source_turbines = ColumnDataSource(df_turbines_short)

In [17]:
output_file("Wind_turbines.html")

# See https://wiki.openstreetmap.org/wiki/Tile_servers for various tile services
MAP_TILES = {"OpenMap": WMTSTileSource(url="http://c.tile.openstreetmap.org/{Z}/{X}/{Y}.png"),
         "ESRI": WMTSTileSource(url="https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{Z}/{Y}/{X}.jpg"),
         "OpenTopoMap": WMTSTileSource(url="https://tile.opentopomap.org/{Z}/{X}/{Y}.png")}

# Create a bokeh figure with tiles
plot_map = figure(width=800, 
                  height=800,
                  x_axis_type="mercator", 
                  y_axis_type="mercator",
                )

plot_map.add_tile(MAP_TILES['ESRI'])


TileRenderer(id='p1035', ...)

In [18]:
marker_options = {
    "marker": "circle_y",
    "line_width": 1,
    "alpha": 0.8,
    "fill_color": "data_rank_colour",
    "line_color": "white",
    "legend_group": "data_rank",
}

render_hydro = plot_map.scatter(x="x", y="y", source=source_turbines, size=8, **marker_options)

hover_hydro = HoverTool(renderers=[render_hydro])
hover_hydro.tooltips = [("Hub height", "@height_hub"),
                        ("Rotor diameter", "@rotor_diameter"),
                        ("Capacity", "@generator_output_electricity"),
                        ("Model", "@model"),
                        ("Location", "@coordinates")]

plot_map.add_tools(hover_hydro)

render_hydro.selection_glyph = None
render_hydro.nonselection_glyph = None

url = "https://www.openstreetmap.org/node/@id/"
render_hydro.selection_glyph = None
render_hydro.nonselection_glyph = None
taptool_hydro = TapTool(renderers=[render_hydro])
taptool_hydro.callback = OpenURL(url=url)
plot_map.add_tools(taptool_hydro)

In [19]:
from bokeh.models import Range1d

# x,y = TRANSFORM_4326_TO_3857.transform([-40,20],[-20,55])

# # set a range using a Range1d
# plot_map.x_range = Range1d(x[0], x[1])
# plot_map.y_range = Range1d(y[0], y[1])

plot_map.title.text = "Wind Turbines from OpenStreetMap"

In [20]:
show(plot_map)

In [33]:
turbines = df_turbines_short[df_turbines_short["data_rank"]=="silver"]

In [35]:
turbines = turbines.astype({"rotor_diameter": float, "height_hub": float})

In [39]:
turbineModels = {}
for cnt,turbine in turbines.iterrows():

    print(turbine)

    turbineModel = f'RD{turbine["rotor_diameter"]:0.0f} HH{turbine["height_hub"]:0.0f}'

    print(turbineModel)

    turbineModels[turbineModel]=turbine

    turbines.loc[cnt,"turbineModel"]=turbineModel


source                                               NaN
generator_output_electricity                        3 MW
start_date                                           NaN
height_hub                                          78.0
generator_type                           horizontal_axis
rotor_diameter                                      82.0
manufacturer                                     ENERCON
model                                                NaN
lat                                            38.189762
lon                                              23.2201
id                                            3643973616
x                                         2584849.697037
y                                         4606267.568612
coordinates                     (38.1897617, 23.2200999)
data_rank                                         silver
data_rank_colour                                  silver
turbineModel                                   RD82 HH78
Name: 113, dtype: object
RD82 H

In [40]:
# print turbine models
for turbine in turbineModels:
    print(turbineModels[turbine])

source                                               NaN
generator_output_electricity                        3 MW
start_date                                           NaN
height_hub                                          78.0
generator_type                           horizontal_axis
rotor_diameter                                      82.0
manufacturer                                     ENERCON
model                                                NaN
lat                                            38.191851
lon                                            23.214996
id                                            9895203447
x                                         2584281.578016
y                                         4606563.474041
coordinates                     (38.1918509, 23.2149964)
data_rank                                         silver
data_rank_colour                                  silver
turbineModel                                   RD82 HH78
Name: 502, dtype: object
source

In [43]:
for turbine in turbineModels:
    
    wt_model = turbineModels[turbine]
    
    # create turbine
    collada_wt_model = collada_wt.create_turbine(tower_height=0.97*wt_model["height_hub"],#95,
                    tower_bot_diameter = 0.04*wt_model["height_hub"],
                    tower_top_diameter = 0.03*wt_model["height_hub"],
                    nacelle_height = 0.03*wt_model["height_hub"],
                    nacelle_length = 0.2*wt_model["height_hub"],
                    nacelle_overhang = 0.08*wt_model["height_hub"],
                    rotor_diameter = wt_model["rotor_diameter"],
                    blade_root_length = 0.01*wt_model["rotor_diameter"],
                    blade_root_diameter = 0.01*wt_model["rotor_diameter"],
                    blade_chord = 0.025*wt_model["rotor_diameter"],
                    blade_tip_size =0.5,
                    blade_twist = 30,                
                    )

    collada_wt_model.write(f"tmp/{turbine}.dae")

In [49]:
turbines.iloc[0].name

113

In [55]:
# create kml for Google Earth
kml = simplekml.Kml()

fol_site = kml.newfolder(name="3D turbines")
fol_labels = fol_site.newfolder(name="labels")
fol_models = fol_site.newfolder(name="models")

for cnt,turbine in turbines.iterrows():       

    # then add the turbine labels to the site labels folder
    label = fol_labels.newpoint(name=turbine.name)
    label.coords = [(turbine["lon"],turbine["lat"])]

    # and add the turbine models to the site model folder
    model = fol_models.newmodel(name=turbine.name)
    model.description = turbine["model"]
    model.link.href = f'{turbine["turbineModel"]}.dae'
    model.altitudemode = "clampToGround"
    model.location.latitude = turbine["lat"]
    model.location.longitude = turbine["lon"]
    model.scale.x = 1
    model.scale.y = 1
    model.scale.z = 1
    model.orientation.heading = 90

kml.save("tmp/portfolio.kml")